# Hiperparaméterek optimalizációja

A paraméteroptimalizációhoz a Bayesi optimalizációs technikát alkalmazom. Ehhez meg kell hívni a szükséges könyvtárat.

In [1]:
#!pip install bayesian-optimization
from bayes_opt import BayesianOptimization
import sys
sys.path.append('c:\\users\\ifjto\\appdata\\local\\programs\\python\\python37\\lib\\site-packages')
import numpy as np
import import_ipynb
from model import ResidualBlock, NeuralNetwork, MyDataset, get_data, train, classify, validate
import torch
from torch import nn
from torch.utils.data import *
import torch.optim as optim
import torch.cuda
import torchvision.transforms as transforms
import cv2
import os
import time
from IPython.display import Image
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt

importing Jupyter notebook from model.ipynb
CUDA available!


## Optimalizálás a nemek felismeréséhez

Az optimalizálandó paraméterek értékkészletét állítsuk be. Természetesen nem megengedhetőek az óriási értékek, mivel nincs rá elegendő számítási kapacitás - áldozatot kell hoznunk. Ezen persze segíthetünk azzal, hogy több kicsi optimalizálást futtatunk, és megpróbáljuk kitalálni, hogy melyik paraméterértékek túl kicsik vagy nagyok, így szűkítve az optimalizáló értékkészletét. Néhány magyarázat a választott határokhoz:
- nFeat: tapasztalatok azt mutatják, hogy ha 1-nek választjuk, akkor lényegesen romlik a háló pontossága
- nLevels: 1 szintű hálónak nincs sok értelme, 6 felett pedig 1x1-es konvolúció lépne fel, ami szintén értelmetlen
- bSize: a választott értéket 2 kitevőjére teszem, így mindig 2 hatványt próbálunk ki. Ez azért jó, mert gyorsabb lehet a tanítás
- a kategorikus változókat kézzel választom ki

In [2]:
pbounds = {'nFeat' : (3, 15),
           'nLevels' : (2, 5),
           'layersPerLevel' : (2, 6),
           'kernelSize' : (3, 8),
           'dropout' : (0, 0.6),
           'bSize' : (4, 6),
           'lr' : (-5.5, -2),
           'decay' : (-9, -3)}

Készítsünk egy függvényt, amit az optimalizálónk tud használni. A függvény felelőssége legyen, hogy csak elfogadható értékekkel híva a train-t.

In [3]:
def gender_train(nFeat, nLevels, layersPerLevel, kernelSize, dropout, bSize, lr, decay):
    nFeat = int(round(nFeat))
    nLevels = int(round(nLevels))
    layersPerLevel = int(round(layersPerLevel))
    kernelSize = int(round(kernelSize)//2*2+1)
    bSize = int(pow(2, round(bSize)))
    numEpoch = 30
    lr = pow(10, lr)
    decay = pow(10, decay)
    
    # redirecting stdout to avoid disturbing prints
    original_out = sys.stdout
    sys.stdout = open("temp.txt", "a")
    
    trainloader, testloader, validationloader, frequencies = get_data("gender", ("Male", "Female"), bSize)
    net, stats, losses = train(2, nFeat, nLevels, layersPerLevel, kernelSize, dropout, trainloader, testloader,
                               lr, decay, numEpoch, frequencies)
    
    # resetting the stdout
    sys.stdout = original_out
    return stats[-1]

Optimalizáljunk. A jó eredményhez kezdjük néhány random lépéssel a próbálkozást.

In [4]:
start_time = time.time()
optimizer = BayesianOptimization(
    f=gender_train,
    pbounds=pbounds,
    random_state=1,
)

optimizer.maximize(
    init_points=5,
    n_iter=20,
)

print(optimizer.max)
print('Finished after ', round(time.time() - start_time), " seconds")

|   iter    |  target   |   bSize   |   decay   |  dropout  | kernel... | layers... |    lr     |   nFeat   |  nLevels  | numEpoch  |
-------------------------------------------------------------------------------------------------------------------------------------
|  1        |  0.4776   |  4.834    | -4.678    |  6.862e-0 |  4.512    |  2.587    | -7.446    |  4.304    |  3.037    |  22.76    |
|  2        |  0.5038   |  5.078    | -6.485    |  0.4111   |  4.022    |  5.512    | -7.836    |  7.693    |  3.252    |  24.7     |
|  3        |  0.8822   |  4.281    | -7.811    |  0.4804   |  7.841    |  3.254    | -3.846    |  9.135    |  4.684    |  19.02    |
|  4        |  0.7766   |  4.078    | -7.981    |  0.5269   |  3.492    |  3.684    | -2.253    |  6.732    |  4.076    |  21.79    |
|  5        |  0.8046   |  5.373    | -3.992    |  0.01097  |  6.751    |  5.955    | -3.511    |  4.963    |  4.368    |  19.24    |
|  6        |  0.662    |  6.0      | -3.0      |  0.0      | 

## Optimalizálás a rasszok felismeréséhez

Az előzőkhöz hasonlóan járunk el, de az értékkészleten változtatunk, ha kell.

In [3]:
pbounds = {'nFeat' : (19, 21),
           'nLevels' : (2, 3),
           'layersPerLevel' : (2, 3),
           'kernelSize' : (3, 9),
           'dropout' : (0.05, 0.5),
           'bSize' : (4, 5),
           'lr' : (-4.3, -3.5),
           'decay' : (-9, -3)}

def race_train(nFeat, nLevels, layersPerLevel, kernelSize, dropout, bSize, lr, decay):
    nFeat = int(round(nFeat))
    nLevels = int(round(nLevels))
    layersPerLevel = int(round(layersPerLevel))
    kernelSize = int(round(kernelSize)//2*2+1)
    bSize = int(pow(2, round(bSize)))
    numEpoch = 30
    lr = pow(10, lr)
    decay = pow(10, decay)
    
    # redirecting stdout to avoid disturbing prints
    original_out = sys.stdout
    sys.stdout = open("temp.txt", "a")
    
    accepted_values = ("Black", "White", "Asian")
    trainloader, testloader, validationloader, frequencies = get_data("race", accepted_values, bSize)
    net, stats, losses = train(len(accepted_values), nFeat, nLevels, layersPerLevel, kernelSize, dropout, trainloader,
                               testloader, lr, decay, numEpoch, frequencies)
    
    # resetting the stdout
    sys.stdout = original_out
    return stats[-1]

És optimalizáljunk.

In [4]:
start_time = time.time()
optimizer = BayesianOptimization(
    f=race_train,
    pbounds=pbounds,
    random_state=1,
)

optimizer.maximize(
    init_points=10,
    n_iter=65,
)

print(optimizer.max)
print('Finished after ', round(time.time() - start_time), " seconds")

|   iter    |  target   |   bSize   |   decay   |  dropout  | kernel... | layers... |    lr     |   nFeat   |  nLevels  |
-------------------------------------------------------------------------------------------------------------------------
|  1        |  0.3604   |  4.417    | -4.678    |  5.719e-0 |  4.814    |  2.294    | -4.751    |  16.12    |  2.346    |
|  2        |  0.3492   |  4.397    | -5.767    |  0.2096   |  7.111    |  2.409    | -2.629    |  15.16    |  2.67     |
|  3        |  0.3503   |  4.417    | -5.648    |  0.07019  |  4.189    |  3.601    | -2.386    |  16.88    |  2.692    |
|  4        |  0.4913   |  4.876    | -3.632    |  0.04252  |  3.234    |  2.34     | -2.629    |  15.59    |  2.421    |
|  5        |  0.3333   |  4.958    | -5.801    |  0.3459   |  4.893    |  3.373    | -2.747    |  15.11    |  2.75     |
|  6        |  0.5294   |  4.989    | -4.511    |  0.1402   |  7.736    |  2.206    | -3.791    |  20.45    |  2.294    |
|  7        |  0.4854   

## Optimalizálás a munkakörök felismeréséhez

In [2]:
pbounds = {'nFeat' : (5, 21),
           'nLevels' : (2, 5),
           'layersPerLevel' : (2, 5),
           'kernelSize' : (3, 9),
           'dropout' : (0, 0.5),
           'bSize' : (4, 5),
           'lr' : (-8, -2),
           'decay' : (-9, -3)}

def occupation_train(nFeat, nLevels, layersPerLevel, kernelSize, dropout, bSize, lr, decay):
    nFeat = int(round(nFeat))
    nLevels = int(round(nLevels))
    layersPerLevel = int(round(layersPerLevel))
    kernelSize = int(round(kernelSize)//2*2+1)
    bSize = int(pow(2, round(bSize)))
    numEpoch = 30
    lr = pow(10, lr)
    decay = pow(10, decay)
    
    # redirecting stdout to avoid disturbing prints
    original_out = sys.stdout
    sys.stdout = open("temp.txt", "a")
    
    accepted_values = ("Star", "Sciences", "Sports", "Arts", "Business/politics", "Military", "Religion", 
                  "Crime", "Music", "Law")
    trainloader, testloader, validationloader, frequencies = get_data("occupation_category", accepted_values, bSize)
    net, stats, losses = train(len(accepted_values), nFeat, nLevels, layersPerLevel, kernelSize, dropout, trainloader,
                               testloader, lr, decay, numEpoch, frequencies)
    
    # resetting the stdout
    sys.stdout = original_out
    return stats[-1]

In [3]:
start_time = time.time()
optimizer = BayesianOptimization(
    f=occupation_train,
    pbounds=pbounds,
    random_state=1,
)

optimizer.maximize(
    init_points=5,
    n_iter=65,
)

print(optimizer.max)
print('Finished after ', round(time.time() - start_time), " seconds")

|   iter    |  target   |   bSize   |   decay   |  dropout  | kernel... | layers... |    lr     |   nFeat   |  nLevels  |
-------------------------------------------------------------------------------------------------------------------------
|  1        |  0.1104   |  4.417    | -4.678    |  5.719e-0 |  4.814    |  2.44     | -7.446    |  7.98     |  3.037    |
|  2        |  0.1401   |  4.397    | -5.767    |  0.2096   |  7.111    |  2.613    | -2.731    |  5.438    |  4.011    |
|  3        |  0.1534   |  4.417    | -5.648    |  0.07019  |  4.189    |  4.402    | -2.19     |  10.01    |  4.077    |
|  4        |  0.1644   |  4.876    | -3.632    |  0.04252  |  3.234    |  2.509    | -2.731    |  6.574    |  3.263    |
|  5        |  0.1476   |  4.958    | -5.801    |  0.3459   |  4.893    |  4.06     | -2.992    |  5.293    |  4.25     |
|  6        |  0.1673   |  4.682    | -9.0      |  0.0362   |  3.0      |  2.0      | -2.0      |  5.96     |  2.798    |
|  7        |  0.1437   

|  66       |  0.1507   |  5.0      | -6.579    |  0.5      |  7.618    |  4.689    | -3.309    |  12.93    |  2.449    |
|  67       |  0.1772   |  4.063    | -3.238    |  0.3533   |  8.912    |  2.378    | -4.354    |  20.89    |  2.48     |
|  68       |  0.1364   |  4.044    | -8.274    |  0.3804   |  7.607    |  2.39     | -2.595    |  20.83    |  4.887    |
|  69       |  0.1317   |  4.201    | -3.489    |  0.172    |  3.054    |  4.808    | -5.395    |  20.64    |  4.75     |
|  70       |  0.1472   |  4.0      | -9.0      |  0.5      |  3.0      |  2.0      | -2.0      |  21.0     |  2.0      |
{'target': 0.2034742849971351, 'params': {'bSize': 4.021404677708202, 'decay': -8.669600781396264, 'dropout': 0.10388419665685261, 'kernelSize': 8.648116853863876, 'layersPerLevel': 3.1154707291179657, 'lr': -4.634232320937716, 'nFeat': 20.87164490006776, 'nLevels': 4.765290329759951}}
Finished after  81521  seconds
